!['Ex2'](img/ex2.png)

In [52]:
P = [9, 16*10**3, 15.9*10**3, 15.2*10**3, 15*10**3, 10] #kPa
T1, T3, T4, T5 = 38, 35, 626, 600  #C
fluido = 'Water'
n_pump, n_turb = 0.85, 0.87
dmdt = 15 #kg/s

In [40]:
from CoolProp.CoolProp import State as st

In [41]:
st1 = st(fluido, {'P': P[0], 'T': T1 + 273.15})
st3 = st(fluido, {'P': P[2], 'T': T3 + 273.15})
st4 = st(fluido, {'P': P[3], 'T': T4 + 273.15})
st5 = st(fluido, {'P': P[4], 'T': T5 + 273.15})

$$ st_{2s}: st_{2s}(P_{2}, S_{1}) $$
$$ h_{2} = \frac{h_{2s} - h_{1}}{\eta_{pump}} + h_{1} $$
$$ st_{2}: st_{2}(P_{2}, h_{2}) $$

In [42]:
st2s = st(fluido, {'P': P[1], 'S': st1.s})
h2 = st1.h + (st2s.h - st1.h)/n_pump
st2 = st(fluido, {'P': P[1], 'H': h2})

$$ st_{6s}: st_{6s}(P_{6}, S_{5}) $$
$$ h_{6} = h_{5} - \eta_{turb}(h_{5} - h_{6s}) $$
$$ st_{6}: st_{6}(P_{6}, h_{6}) $$

In [43]:
st6s = st(fluido, {'P': P[5], 'S': st5.s})
h6 = st5.h - n_turb*(st5.h - st6s.h)
st6 = st(fluido, {'P': P[5], 'H': h6})

In [44]:
from numpy import arange, absolute

In [45]:
estados = st1, st2, st3, st4, st5, st6

pressao, temperatura, entalpia, entropia, titulo, qtd_sts = [], [], [], [], [], arange(1, 7, 1)

for propriedade in estados:
    pressao.append(round(propriedade.p, 2))
    temperatura.append(round(propriedade.T - 273.15, 2))
    entalpia.append(round(propriedade.h, 2))
    entropia.append(round(propriedade.s, 2))

    if absolute(round(propriedade.Q, 2)) == 1:
        titulo.append('Vap. Sat.')
    elif propriedade.Q == 0:
        titulo.append('Liq. Sat.')
    else:
        titulo.append(round(propriedade.Q, 2))


In [46]:
from pandas import DataFrame

In [47]:
data_st = {
    'Estado': qtd_sts,
    'Temperatura [ºC]': temperatura,
    'Pressão [kPa]': pressao,
    'Entalpia [kJ/kg]': entalpia,
    'Entropia [kJ/kg/K]': entropia,
    'Titulo [-]': titulo
}

df_st = DataFrame(data_st).style.hide(axis='index').format(precision=2)

In [51]:
display(data_st)

{'Estado': array([1, 2, 3, 4, 5, 6]),
 'Temperatura [ºC]': [38.0, 39.13, 35.0, 626.0, 600.0, 45.81],
 'Pressão [kPa]': [9.0, 16000.0, 15900.0, 15200.0, 15000.0, 10.0],
 'Entalpia [kJ/kg]': [159.18, 178.06, 160.85, 3648.96, 3583.13, 2306.06],
 'Entropia [kJ/kg/K]': [0.55, 0.55, 0.5, 6.75, 6.68, 7.28],
 'Titulo [-]': ['Vap. Sat.',
  'Vap. Sat.',
  'Vap. Sat.',
  'Vap. Sat.',
  'Vap. Sat.',
  0.88]}

$$ q_{in} = h_{4} - h_{3} $$
$$ q_{out} = h_{6} - h_{1} $$
$$ \eta_{th} = 1 - \frac{q_{out}}{q_{in}} $$
$$ \omega_{turb, out} = \eta_{turb}(h_{5} - h_{6}) $$
$$ \omega_{pump, in} = \eta_{pump}(h_{2} - h_{1}) $$
$$ W_{out} = \frac{dm}{dt}\omega_{turb, out} $$

In [53]:
q_in = st4.h - st3.h
q_out = st6.h - st1.h
nth = 100*(1 - q_out/q_in)
w_turb_out = n_turb*(st5.h - st6.h)
w_pump_in = n_pump*(st2.h - st1.h)
W_out = dmdt*w_turb_out

In [55]:
print(f' q_in: {q_in:.2f} kJ/kg \n', 
      f'q_out: {q_out:.2f} kJ/kg\n',
      f'n_th: {nth:.2f}% \n',
      f'w_turb_out: {w_turb_out:.2f} kJ/kg \n',
      f'w_pump_in: {w_pump_in:.2f} kJ/kg \n',
      f'W_out: {W_out:.2f} kW')

 q_in: 3488.11 kJ/kg 
 q_out: 2146.89 kJ/kg
 n_th: 38.45% 
 w_turb_out: 1111.05 kJ/kg 
 w_pump_in: 16.05 kJ/kg 
 W_out: 16665.73 kW
